In [1]:
import re

import pandas as pd
import requests 

**Part 2b.1: Scrape webpage for data links**

In [2]:
r = requests.get('http://data.kk.dk/dataset/faste-trafiktaellinger')
html = r.text

We find all urls with the regex code used below. This was found by Googling and discovering [this webpage](https://stackoverflow.com/questions/6883049/regex-to-find-urls-in-string-in-python))

In [3]:
re_url = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
urls = re.findall(re_url, html)

We select the urls that link to MS Excel files

In [4]:
urls_w_data = [u for u in urls if u.endswith('.xlsx')]

In [5]:
urls_w_data

['http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx',
 'http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/0bfc0425-0b2f-43b4-be71-117b4265598d/download/faste-trafiktaellinger-2006.xlsx',
 'http://data.kk.dk/storage/f/2015-05-11T07%3A35%3A06.512Z/faste-trafiktaellinger-2007.xlsx',
 'http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/e4f321ad-76bc-483f-839e-e04cfea6bfc0/download/faste-trafiktaellinger-2008.xlsx',
 'http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/bac5c582-bdd0-45db-b174-f835e82851bc/download/faste-trafiktaellinger-2009.xlsx',
 'http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/a209aef9-802c-4cda-9b42-f4364877b04f/download/faste-trafiktaellinger-2010.xlsx',
 'http://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/8b2f60a9-17ac-437d-ae6e-ad44c0a49edd/download/faste-trafi

We load the data using pandas' Excel reader. Details are found below line by line.

In [6]:
# loading the datasets into a list using list comprehension
# note that we skip the first 10 lines as they are not useful
data_sets = [pd.read_excel(u, skiprows=10) for u in urls_w_data]

In [7]:
data_raw = pd.concat(data_sets)

**Part 2b.2: Structure your dataset**

In [8]:
data_idx = data_raw.reset_index(drop=True)

In [9]:
data_idx.drop('Spor', axis=1, inplace=True)

In [10]:
dk_to_uk = {'Vejnavn':'road_name',
            '(UTM32)':'UTM32_east',
            '(UTM32).1':'UTM32_north',
            'Dato':'date',
            'Vej-Id':'road_id'}

data_long = data_idx\
                .rename(columns={'index':'year'})\
                .rename(columns=dk_to_uk)

In [11]:
data_long = data_long.copy()

del data_sets, data_raw, data_idx

**Part 2b.3: String data, selection and rotation**

In [12]:
data_long['total'] = data_long.road_id.str[-1] == 'T'

In [13]:
data_long = data_long[data_long.total]

In [14]:
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']
data_geo = data_long[spatial_columns].copy()

In [15]:
data_geo.drop_duplicates(inplace=True)

In [16]:
id_columns = ['road_name', 'date']
hour_columns = [c for c in data_long.columns if 'kl.' in c]

data = pd.melt(data_long, 
               id_vars=id_columns, 
               value_vars=hour_columns,
               var_name='hour_period',
               value_name='traffic')\
         .copy()

del data_long

In [17]:
pd.options.display.max_rows = 20
data

,road_name,date,hour_period,traffic
0,Ellebjergvej,01.04.2005,kl.00-01,191.0
1,Ellebjergvej,02.04.2005,kl.00-01,381.0
2,Ellebjergvej,03.04.2005,kl.00-01,457.0
3,Ellebjergvej,04.04.2005,kl.00-01,140.0
4,Ellebjergvej,05.04.2005,kl.00-01,159.0
5,Ellebjergvej,06.04.2005,kl.00-01,173.0
6,Ellebjergvej,07.04.2005,kl.00-01,190.0
7,Ellebjergvej,08.04.2005,kl.00-01,211.0
8,Ellebjergvej,09.04.2005,kl.00-01,366.0
9,Ellebjergvej,10.04.2005,kl.00-01,445.0


In [18]:
data.road_name = data.road_name.astype('category')

**Part 2b.4: Structure temporal data**

In [19]:
data['hour'] = data.hour_period.str[3:5]

In [20]:
time_str = data.date + '-' + data.hour              
data['time'] = pd.to_datetime(time_str, format='%d.%m.%Y-%H')

data.drop(['hour', 'date', 'hour_period'], 1, inplace=True)

In [21]:
data['week_day'] = data.time.dt.weekday

In [22]:
print(data.time.dt.week.head(3))
print(data.time.dt.month.head(3))

0    13
1    13
2    13
Name: time, dtype: int64
0    4
1    4
2    4
Name: time, dtype: int64
